In [1]:
#!/usr/bin/env python  
# -*- coding: utf8 -*- 
     
import logging 
import warnings  
     
     
logging.basicConfig(level=logging.INFO) 
 
 
def filterwarn(): 
    # warnings.simplefilter('ignore', UserWarning) 
    # advance warnings flter function 
    warnings.filterwarnings('ignore', '.*warn.*', UserWarning, 'module') 
 
 
def main(): 
    filterwarn() 
    # compare the two following items. 
    warnings.warn("This is a warning message.") 
    logging.warn("[+] This is a warn message.") 
 
    logging.info("[+] This is a info message.") 
 
if __name__ == "__main__": 
    main()

D:\Program_Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: This is a warning message.
D:\Program_Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
INFO:root:[+] This is a info message.


In [2]:
text = "projectName:""test"" AND body:""Hello"""

In [3]:
from urllib import parse

#这个是js的结果
# encodeURIComponent('中国')
# "%E4%B8%AD%E5%9B%BD"
#jsRet='%E4%B8%AD%E5%9B%BD'
#print(parse.unquote(jsRet))       #输出：中国
#print(jsRet==parse.quote('中国'))  #输出：True
print(parse.quote("projectName:\"test\" AND (body:\"FAILURE_DB\")"))

projectName%3A%22test%22%20AND%20%28body%3A%22FAILURE_DB%22%29


In [69]:
from Utils import utilIdentifier
import importlib
importlib.reload(utilIdentifier)

def identifyText(text):
    text = utilIdentifier.identifyIP(text)
    text = utilIdentifier.identifyDatetime(text)
    text = utilIdentifier.identifyUri(text)
    #text = utilIdentifier.identifyNumber(text), 由后面的token去掉
    return text

In [71]:
import spacy
import string

text = "192.168.1.100 fe80::20c:29ff:fe75:f519/64 2018-03-13 16:27:24 +0900 [warn]: dest=http://github.com/obdg/plda.git #0 failed to flush the buffer fe80::20c:29ff:fe75:f519/64 . retry_time=2 next_retry_seconds=2018-03-13 16:27:54 +0900 chunk=""56745277a4532957f8c4fe9e070b75d1"" error_class=NoMethodError error=""undefined method `has_key?' for #<String:0x007ff21c6d91b8>"

text = identifyText(text)
print(text)
#https://spacy.io/usage/processing-pipelines
#https://github.com/explosion/spaCy/issues/1837
parser = spacy.load('en', disable=['parser', 'ner'])
tokens = parser(text)
for tok in tokens:
    print(tok, tok.tag_, tok.lemma_)
    
#for ent in tokens.ents:
#    print(ent, ent.label, ent.lable_)

IP_TYPE IP_TYPE DATETIME_TYPE [warn]: dest=URL_TYPE #0 failed to flush the buffer IP_TYPE . retry_time=2 next_retry_seconds=DATETIME_TYPE chunk=56745277a4532957f8c4fe9e070b75d1 error_class=NoMethodError error=undefined method `has_key?' for #<String:0x007ff21c6d91b8>
IP_TYPE NNP ip_type
IP_TYPE NNP ip_type
DATETIME_TYPE NNP datetime_type
[ -LRB- [
warn VBP warn
] -RRB- ]
: : :
dest JJS d
= SYM =
URL_TYPE NNS url_type
# $ #
0 CD 0
failed VBD fail
to TO to
flush VB flush
the DT the
buffer NN buffer
IP_TYPE NN ip_type
. . .
retry_time=2 CD retry_time=2
next_retry_seconds NNS next_retry_second
= SYM =
DATETIME_TYPE NNS datetime_type
chunk=56745277a4532957f8c4fe9e070b75d1 VBP chunk=56745277a4532957f8c4fe9e070b75d1
error_class NN error_class
= SYM =
NoMethodError NN nomethoderror
error NN error
= SYM =
undefined JJ undefined
method NN method
` '' `
has_key NN has_key
? . ?
' '' '
for IN for
# $ #
< XX <
String:0x007ff21c6d91b8 XX string:0x007ff21c6d91b8
> XX >


In [72]:
import re
text=""" 
StudentException: Error finding students
        at StudentManager.findStudents(StudentManager.java:13)
        at StudentProgram.main(StudentProgram.java:9)
Caused by: DAOException: Error querying students from database
        at StudentDAO.list(StudentDAO.java:11)
        at StudentManager.findStudents(StudentManager.java:11)
        ... 1 more
Caused by: java.sql.SQLException: Syntax Error
        at DatabaseUtils.executeQuery(DatabaseUtils.java:5)
        at StudentDAO.list(StudentDAO.java:8)
        ... 2 more
"""
from Utils import utilIdentifier
import importlib
importlib.reload(utilIdentifier)

m1 =re.search(r"Caused by:.*", text)
m2 = re.search(r"at.*\(.*.java:\d+\)", text)
if m1:
    print(m1.group(0))
if m2:
    print(m2.group(0))
        
if utilIdentifier.isJavaStackTrace(text):
    print("yes")


Caused by: DAOException: Error querying students from database
at StudentManager.findStudents(StudentManager.java:13)
yes
